In [40]:
import sys
import os
import requests
import json
import csv
import decimal
import copy

#sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartenv/setup/challenges/util/__init__.py'>

In [68]:
!pwd

/smartenv/setup/challenges


In [42]:
SMARTENV_PATH = "/smartenv/"
GEN_CONFIG_PATH = "/smartenv/setup/generated_config/"
GLOBAL_INFO_PATH = GEN_CONFIG_PATH + "/global_info/"
GENESIS_JSON = "/smartenv/setup/environment/genesis_config/go-ethereum/berlin/genesis.json"

PARTICIPANTS_PATH = "/smartenv/participants/"
PARTICIPANTS_CSV = PARTICIPANTS_PATH + "participants.csv"
PP_CSV = PARTICIPANTS_CSV

SCRIPTS_PATH = "/smartenv/util/scripts/"
CHALLENGES_PATH = "/smartenv/challenges/"
CHL_NOTAWALLET_PATH = "/smartenv/challenges/notawallet/"

In [43]:
SRVIP = "172.18.0.8"
SRVPORT = "8545"
SRVCHAINID = "20211102"
SRVNETVER = "25519"

In [44]:
SEED_VALUE = 16*10**18
SEED_VALUE_NOTAWALLET = 10*10**18
SEED_VALUE_BADPARITY = 30*10**18
SEED_VALUE_DAODOWN = 30*10**18
SEED_VALUE_FAILDICE = 30*10**18

# Challenge setup geth

## Start environment: smartenv-geth & smartenv-web3py 

We will run the server/geth node as well as the smar>tenv-web3py client docker containers on the same host machine. 

```bash
$ make run-smartenv-web3py # the container running this notebook
$ make run-smartenv-geth # the geth client this notebook connectes to
```


## Connect and check chain state

Note that for *PoA* nodes the middleware needs to be added.
Therefore, set the `poa` flag to true when connecting. 

In [45]:
#w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.1:8545"))
#w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True)
# check if connection was successful
assert w3.isConnected()

In [46]:
w3.api #web3 version used

'5.20.0'

In [47]:
w3.clientVersion # remote client version to which w3 is connected 

'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.15.4'

In [48]:
print(w3.net.version) # in the console: admin.nodeInfo
assert w3.net.version == SRVNETVER

25519


In [49]:
print(w3.eth.chainId) # in the console: admin.nodeInfo
assert w3.eth.chainId == int(SRVCHAINID)

20211102


In [50]:
w3.eth.blockNumber

15085

In [51]:
w3.eth.getBlock(w3.eth.blockNumber)["gasLimit"]

8000000

In [52]:
w3.eth.mining

False

In [53]:
w3.eth.syncing

False

### Networ check

In [54]:
w3.net.peerCount

1

In [55]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://82ab770c3374e2f8083ce2df1fdc502998d14aa6fac6d39251d738bedd5074ec41dac345eb5927e1711ee5d2af38a7c78e817be4d64495cc71a72c4b25d785b8@131.130.126.71:30303',
  'id': 'da1534301cbcb1c786a1c00b32a3592e9d89503e915c60892b8a3d86d4256319',
  'name': 'Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.15.4',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.18.0.8:47062',
   'remoteAddress': '131.130.126.71:30303',
   'inbound': False,
   'trusted': False,
   'static': False}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 30169,
    'head': '0x9447a196b163b799fe45c069f8116e3e5287cf4ff76d6ff5af5a72ba6edfb17c'}),
   'snap': AttributeDict({'version': 1})})})]

In [56]:
w3.net.listening

True

In [57]:
w3.eth.accounts

['0xb1Cef50494b3C82Aa199DCedD212D333C9265697',
 '0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0',
 '0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1']

### Export (correct) enode info for server (docker container)

Delete the old generated config dir beforehand:

In [58]:
GEN_CONFIG_PATH

'/smartenv/setup/generated_config/'

In [59]:
!mkdir -p {GEN_CONFIG_PATH} 

In [60]:
poa_node_con = False # if connected directly to the poa node switch to True

enode_file = GLOBAL_INFO_PATH + "/enodes"
# only needed if running directly on PoA node server:
#invalid_ip_port = "172.18.0.5:30303"
invalid_ip_port = "127.0.0.1:30303"
valid_ip_port = "128.131.169.35:30303"

if poa_node_con:
    if w3.clientVersion.startswith('Parity'):
        enode = w3.parity.enode()
    elif w3.clientVersion.startswith('Geth'):
        enode = w3.geth.admin.nodeInfo()['enode']
    enode = enode.replace(invalid_ip_port,valid_ip_port)
else:
    enode = w3.geth.admin.peers()[0]["enode"]
enode

'enode://82ab770c3374e2f8083ce2df1fdc502998d14aa6fac6d39251d738bedd5074ec41dac345eb5927e1711ee5d2af38a7c78e817be4d64495cc71a72c4b25d785b8@131.130.126.71:30303'

In [61]:
# Write enode to file:
!mkdir -p $GLOBAL_INFO_PATH
!echo $enode > $enode_file

In [62]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000d6abfd59de4b5c832f74d8f0ab9fd844e0ad02550000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0x4e27219a6b230d7b16326006da6ae29a33d350d35070d9f0eefa58d6a06e7c4b'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

Copy password file over for generated accounts:

In [63]:
!cp {SMARTENV_PATH}/passwordfile {GLOBAL_INFO_PATH}

Copy `genesis.json` over for generated config:

In [64]:
!cp {GENESIS_JSON} {GLOBAL_INFO_PATH}

## Import students

Dowload and prepare studentd csv file:
```
$ cat participants_students_original.csv | awk -F, '{print $1","$2","$4","$3}' > participants_students.csv
$ cat participants_template.csv participants_students.csv participants_placeholder.csv > participants.csv
```

In [92]:
print(PARTICIPANTS_PATH)
print(PARTICIPANTS_CSV)
print(PP_CSV)

/smartenv/participants/
/smartenv/participants/participants.csv
/smartenv/participants/participants.csv


In [93]:
!ls -la {PARTICIPANTS_CSV}

lrwxrwxrwx 1 smartenv smartenv 24 Jun 28 18:43 /smartenv/participants/participants.csv -> participants_current.csv


In [94]:
!head -n2 $PARTICIPANTS_CSV

Aljosha,Judmayer,ajudmayer@sba-research.org,1
Nicholas,Stifter,nstifter@sba-research.org,2


In [95]:
!file $PARTICIPANTS_CSV # get file format/encoding

/smartenv/participants/participants.csv: symbolic link to participants_current.csv


In [96]:
!wc -l $PARTICIPANTS_CSV

19 /smartenv/participants/participants.csv


In [97]:
wc_out = !wc -l $PARTICIPANTS_CSV # check number of stutend and compare later 
pp_count_wc = int(wc_out[0].split(" ")[0])
pp_count_wc

19

In [98]:
pp_list = list() # list of participants
pp_count = 0 # number of participants

in_fieldnames = ['firstname', 'lastname','email','ppid'] # ppid could be matr.

In [99]:
def read_students(path,pp_list):
    with open(path,'r',encoding='utf-8') as f_input:
    #with open(path,'r',encoding='utf-16') as f_input:
    #with open(path,'r',encoding='latin-1') as f_mail:
        #sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter='\t',quotechar='\"')
        sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter=',',quotechar='\"')
        for i,row in enumerate(sreader):
            #print(i,row)
            pp_dict = dict()
            for field in in_fieldnames:
                    pp_dict[field]=row[field]
            pp_list.append(pp_dict)
            #print(students_list)
    pp_count = i+1
    return pp_count

In [100]:
read_students(PP_CSV,pp_list)

19

In [101]:
pp_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1'}

In [102]:
pp_list[-1]

{'firstname': 'placeholder',
 'lastname': 'placeholder',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '7'}

In [103]:
# pp_list # long output

In [104]:
pp_count = len(pp_list)
assert pp_count == pp_count_wc
pp_count

19

## Create keys

In [105]:
SCRIPTS_PATH

'/smartenv/util/scripts/'

In [106]:
!cat {SCRIPTS_PATH}create_accounts.sh 

#!/bin/bash
#
# Run from root of repository

set -e

if [ -z "${1}" ] && [ -z "${PARTICIPANTS+x}" ];
then
    echo "PARTICIPANTS = No number given e.g., '42'"
    exit 2
elif [ ! -z "${1}" ];
then
    export PARTICIPANTS="${1}"
fi
echo "PARTICIPANTS = ${PARTICIPANTS}"

if [ -z "${DATADIR+x}" ];
then
    echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., './datadir/alice/'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

if [ -z "${TARGETDIR+x}" ];
then
        echo "TARGETDIR environment variable missing"
	echo "No Datadir given e.g., './setup/generated_config/keystore'"
	exit 2
fi
echo "TARGETDIR = ${TARGETDIR}"

mkdir -p ${TARGETDIR}

#for i in {0..90}};
for i in $(seq 1 ${PARTICIPANTS});
do 
	echo $i
	bash ./util/scripts/geth_account.sh new
done

cp -r "${DATADIR}/keystore" ${TARGETDIR}


In [107]:
!cat {SCRIPTS_PATH}geth_account.sh

#!/bin/bash
#
#
# Run with docker from root of repository:
# $ PWFILE=./passwordfile DATADIR=./datadir/accountmgt/ bash geth_account.sh new

set -e

if [ -z "${PWFILE+x}" ];
then
	echo "PWFILE environment variable missing"
	echo "No Password file given using default './passwordfile'"
	export PWFILE='./passwordfile'
fi
echo "PWFILE = ${PWFILE}"

if [ -f "${PWFILE}" ];
then
	echo "using stored password ... "	
else
	#pwgen -s -B 32 1 > ${PWFILE}
	echo "ERROR: No passwordfile found!"
	exit 1
fi

if [ -z "${DATADIR+x}" ];
then
        echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., './datadir/bob'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

mkdir -p ${DATADIR}

# run container without network
# just to generate and manage keys
docker run \
	--mount type=bind,source=$(pwd),target=/smartenv \
	-it smartenv-geth:latest \
		geth \
		--datadir "${DATADIR}" \
		--nodiscover \
		--maxpeers 0 \
		--ipcdisable \
		--verbosity 6  \
		--password "${PWFILE}" \
		account "$1" "$2"

Run the following command from the root folder of the repository: 
(Take care that there are not more keys under the keystore in DATADIR or the TARGETDIR)

## Assign addresses to users (testaccounts & students)

In [108]:
#### CHANGE ####
 
# path where the raw ethereuem keys are located:
#source_path="./test/keystore/" 
source_path= GEN_CONFIG_PATH + "/keystore"

# Either run with test users or with all students:
#users_list = copy.deepcopy(testers_list)
users_list = copy.deepcopy(pp_list)

#users_list = copy.deepcopy(testers_list)
#users_list.extend(copy.deepcopy(students_list)

#### CHANGE END ####

In [109]:
# print some user as test before assignment
print(len(users_list))
users_list[0]

19


{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1'}

In [110]:
!ls {GEN_CONFIG_PATH}

global_info  keystore


In [111]:
!cd $source_path && pwd && ls | head -n1

/smartenv/setup/generated_config/keystore
UTC--2021-11-05T10-35-42.522469976Z--b4ebf9c576cac2b3c3caf17860bb70a6b02201f3


In [112]:
#!cd $source_path && ls ./ | head -n1 | xargs -Irpl cat rpl

In [113]:
def read_keypairs(path):
    """read keypairs from keystore
    """
    _keypairs = list()
    # read out keystore folder 
    for root,dirs,files in os.walk(path):
        #print("root = {}".format(root))
        #print("dirs = {}".format(dirs))
        for f in files:
            with open(root + "/" + f,"r",encoding="utf-8") as f_input:
                data = json.loads(f_input.read()) # load all key pair data from file
                data["filename"] = f # add file name to data for later
                _keypairs.append(data)
    return _keypairs   

In [114]:
def assign_addresses(keystore_path,users,attach_path="/smartenv/setup/generated_config/attach"):
    """ map keypairs to users and add info to user dict
    This also creates the attach directory. 
    """
    keypairs = read_keypairs(keystore_path)
    assert len(keypairs) >= len(users)
    
    for u in users:
        kp = keypairs.pop()
        u["account"] = kp["address"]
        #u["keypair"] = kp
        #u["key"] = kp["crypto"]["ciphertext"]
        u["filename"] = kp["filename"]
        
        arg_dest = attach_path + "/" + u["account"]
        !mkdir -p $arg_dest
        arg_src = keystore_path + "/" + kp["filename"]
        !cp $arg_src $arg_dest
        u["attach"] = "/tmp/attach/" + u["account"] + ".tar.gz"
        print(".",end=".")
    return users

In [115]:
attach_path= GEN_CONFIG_PATH + "/attach"
#!rm -rf attach_path
assign_addresses(source_path,users_list,attach_path)

# print some user as test after assignment
users_list[0]

......................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz'}

In [116]:
def getBalance(address):
    address = w3.toChecksumAddress(address)
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [117]:
getBalance(users_list[0]["account"])

0

In [118]:
w3.eth.accounts

['0xb1Cef50494b3C82Aa199DCedD212D333C9265697',
 '0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0',
 '0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1']

In [119]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xb1Cef50494b3C82Aa199DCedD212D333C9265697 :  1000000
0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0 :  1000000
0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1 :  1000000


In [120]:
w3.eth.defaultAccount = w3.eth.accounts[0]

In [121]:
w3.eth.defaultAccount

'0xb1Cef50494b3C82Aa199DCedD212D333C9265697'

In [122]:
def send_accounts_money(users,amount):
    for u in users:
        account = w3.toChecksumAddress(u["account"])
        u["seedbalance"] = amount
        w3.eth.sendTransaction({"from":w3.eth.defaultAccount,"to":account,"value":amount})
    return

In [123]:
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"notforproductiveuse",0)

True

In [124]:
send_accounts_money(users_list,SEED_VALUE)

In [125]:
!sleep 16
getBalance(users_list[0]["account"])

Decimal('16')

In [126]:
getBalance(users_list[-1]["account"])

Decimal('16')

In [127]:
def check_accounts_money(users,predicate=(lambda a, b: a == b)):
    for u in users:
        u_balance = w3.eth.getBalance(w3.toChecksumAddress(u["account"]))
        u["balance"] = u_balance
        u_seedbalance = u["seedbalance"]
        if predicate(u_balance,u_seedbalance) == False:
            print("Predicate violation for:\n",u)
            assert False,"Predicate violation"

In [128]:
check_accounts_money(users_list)

In [129]:
check_accounts_money(users_list,predicate=(lambda a, b: a >= b))
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

### Test manual account handling

### Deployment functions

In [130]:
sys.path.append( SMARTENV_PATH + "/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartenv/setup/challenges/util/__init__.py'>

In [131]:
def forall_compile_and_deploy(users,
                              path,
                              contract_name,
                              value,
                              force_argument=None,
                              argument2=None,
                              compiler="solc"):
    print("Started for {} users:".format(len(users)))
    for u in users:
        if force_argument is not None:
            print("*",end="")
            #print("Force argument: {}".format(force_argument))
            u_addr = force_argument
        else:
            u_addr = w3.toChecksumAddress(u["account"])
            
        if argument2 is not None and argument2 is True:
            print("~",end="")
            #print("Double sha3 argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=w3.toInt(hexstr=w3.sha3(text=w3.sha3(text=u_addr).hex()).hex()),
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        elif argument2 is not None and isinstance(argument2,type(str())):
            print("+",end="")
            #print("Custom argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=argument2,
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        else:
            print(".",end="")
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        # returns HexBytes
        # import hexbytes
        # help(hexbytes.main)
        u[contract_name + "_tx"] = tx_hash.hex()
        u[contract_name + "_seedvalue"] = value
        
    for u in users:
        tx_receipt = w3.eth.waitForTransactionReceipt(u[contract_name + "_tx"])
        u[contract_name + "_addr"] = tx_receipt['contractAddress']
        print("-",end="")
        
    return users

In [132]:
def forall_check_contract(users,
                          c_path,
                          c_name,
                          predicate=(lambda a, b: a == b),
                          compiler="solc"):
    # Check for all users if the contract is initialized to the respective user
    # and that the balance is the seed value
    print("Started for {} users:".format(len(users)))
    for u in users:
        c_abi = util.compile_contract_with_libs(compiler_path=compiler,src_path=c_path)["abi"]
        u_c_addr = u[c_name + "_addr"]
        u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=False)
    
        # check owner
        #print(u_c_instance.functions.getStudent().call())
        #print(w3.toChecksumAddress(u["account"]))
        assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(u["account"])   
        # check balance
        u_c_seedvalue = u[c_name + "_seedvalue"]
        u_c_balance = w3.eth.getBalance(u_c_addr)   
        u[c_name + "_balance"] = u_c_balance
        if predicate(u_c_balance,u_c_seedvalue) == False:
            print("Predicate violation for:\n",u)
        print(".",end="")
    return users

In [133]:
def copy_challenge_metadata(users,challenge_path,challenge_name,attach_path="./attach"):
    # copy challenge metadata to individual user folders helper function
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_dest_dir = attach_path + "/" + d + "/" + challenge_name
            print(arg_dest_dir)
            !mkdir -p $arg_dest_dir
            !cp -r $challenge_path $arg_dest_dir

# Challange: notawallet 

In [134]:
CHL_NOTAWALLET_PATH

'/smartenv/challenges/notawallet/'

In [135]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [136]:
# users_list[-1]
users_list[1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nstifter@sba-research.org',
 'ppid': '2',
 'account': '80e3da4b589fc77889dfed3aa33f7dd36d213e32',
 'filename': 'UTC--2021-11-05T10-36-09.390080453Z--80e3da4b589fc77889dfed3aa33f7dd36d213e32',
 'attach': '/tmp/attach/80e3da4b589fc77889dfed3aa33f7dd36d213e32.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [137]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [138]:
!solc --overwrite -o "{CHL_NOTAWALLET_PATH}/publish/" --abi --bin "{CHL_NOTAWALLET_PATH}/publish/NotAWallet.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges/notawallet//publish/.


In [139]:
## Connections refresh maybe needed
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True) 
assert w3.isConnected()

In [140]:
path= CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol"
challange_name="notawallet"
seedvalue=10*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 19 users:
...................-------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B'}

In [141]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [142]:
c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path=CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol")["abi"]
c_abi

[{'inputs': [{'internalType': 'address',
    'name': '_student',
    'type': 'address'}],
  'stateMutability': 'payable',
  'type': 'constructor'},
 {'inputs': [{'internalType': 'address',
    'name': 'newowner',
    'type': 'address'}],
  'name': 'addOwner',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'deposit',
  'outputs': [],
  'stateMutability': 'payable',
  'type': 'function'},
 {'inputs': [],
  'name': 'getStudent',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'address',
    'name': 'testowner',
    'type': 'address'}],
  'name': 'isOwner',
  'outputs': [{'internalType': 'bool', 'name': '', 'type': 'bool'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'owner',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'functi

In [143]:
w3.toChecksumAddress(users_list[0]["account"])

'0x9DdBEe8f1Bb05273118d8e802797e5fB93Fc1BC0'

In [144]:
notawallet_addr = users_list[0]["notawallet_addr"]
notawallet_addr

'0x4c2549e53D28e76cC9FDe242115F0Cd247da786B'

In [145]:
u_c_instance = util.get_contract_instance(notawallet_addr,c_abi,concise=False)

In [146]:
assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(users_list[0]["account"])
u_c_instance.functions.getStudent().call()

'0x9DdBEe8f1Bb05273118d8e802797e5fB93Fc1BC0'

In [147]:
path=CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol"
challange_name="notawallet"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 19 users:
...................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000}

In [148]:
# copy challenge metadata
challenge_path= CHL_NOTAWALLET_PATH + "/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="notawallet"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/166724aa9cc196ba721d86ef06fa61271301e011/notawallet
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/notawallet
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/notawallet
/smartenv/setup/generated_config//attach/12c7b846d3b99747a82244cbac281d9bc914c9f3/notawallet
/smartenv/setup/generated_config//attach/fcfb41d1d12f174606f35e1c9f63c38c7233b61c/notawallet
/smartenv/setup/generated_config//attach/a48b5d4238c163368ccb060388638fb6d98a6ce9/notawallet
/smartenv/setup/generated_config//attach/6b532e866ee6dbe941678081d08b2b874136c66a/notawallet
/smartenv/setup/generated_config//attach/b4ebf9c576cac2b3c3caf17860bb70a6b02201f3/notawallet
/smartenv/setup/generated_config//attach/fb327fb2321e3aa0e221d65758d0c790de01158c/notawallet
/smartenv/setup/generated_config//attach/98970cac400e9b2194d4402064e77031c2ce11bd/notawallet
/smartenv/setup/generated_config//attach/e9db33e3446894eace95a0fcf96b8

# Challenge: badparity

In [149]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000}

In [150]:
!cat "{CHALLENGES_PATH}/badparity/publish/Wallet.sol"

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Wallet {
    address walletLibrary;
    address payable owner;
    address payable student;

    event LogValue(uint256 exitcode,uint256 amount);

    // constructor, called once when this contract is created 
    constructor(address payable _student, address lib) payable {
        student = _student;  
        walletLibrary = lib; // hardcode lib address at deploy time
        // init the owner with the respective lib contract
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
    }

    function getOwner() public view returns (address payable) {
        return owner;
    } 

    function getWalletLibrary() public view returns (address) {
        return walletLibrary;
    }

    function withdraw(uint256 amount) public returns (bool) {
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
        if ( suc

In [151]:
!solc --overwrite -o "{CHALLENGES_PATH}/badparity/publish/" --abi --bin "{CHALLENGES_PATH}/badparity/publish/Wallet.sol"

 --> /smartenv/challenges//badparity/publish/Wallet.sol:4:1:
  |
4 | contract Wallet {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> /smartenv/challenges//badparity/publish/Wallet.sol:47:5:
   |
47 |     fallback() external payable {
   |     ^ (Relevant source part starts here and spans across multiple lines).

  --> /smartenv/challenges//badparity/publish/Wallet.sol:16:9:
   |
16 |         walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
   |         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  --> /smartenv/challenges//badparity/publish/Wallet.sol:50:9:
   |
50 |         walletLibrary.delegatecall(msg.data);
   |         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  --> /smartenv/challenges//badparity/publish/Wallet.sol:28:24:
   |
28 |         (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWith

In [152]:
!solc --overwrite -o "{CHALLENGES_PATH}/badparity/publish/" --abi --bin "{CHALLENGES_PATH}/badparity/publish/WalletLibrary.sol"

 --> /smartenv/challenges//badparity/publish/WalletLibrary.sol:4:1:
  |
4 | contract WalletLibrary {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> /smartenv/challenges//badparity/publish/WalletLibrary.sol:36:6:
   |
36 |      fallback() external payable {
   |      ^ (Relevant source part starts here and spans across multiple lines).



In [153]:
# compile and deploy lib once for all users
path=CHALLENGES_PATH + "/badparity/publish/WalletLibrary.sol"

lib = util.compile_and_deploy_contract(path,
                                 patch_api=False,
                                 concise=False)
lib.address

'0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4'

In [154]:
path=CHALLENGES_PATH + "/badparity/publish/Wallet.sol"
challange_name="badparity"
seedvalue=SEED_VALUE_BADPARITY

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=lib.address)

for u in users_list:
    u[challange_name + "_lib"] = lib.address

users_list[0]

Started for 19 users:
+++++++++++++++++++-------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4'}

In [155]:
# spot check of one contract
c_abi = util.compile_contract_with_libs("solc",path)["abi"]
u_c_addr = users_list[0][challange_name + "_addr"]
u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=True)

assert users_list[0][challange_name + "_lib"] == lib.address
    
# check lib
assert str(u_c_instance.getWalletLibrary()).lower() == str(lib.address).lower()
assert str(u_c_instance.getOwner()).lower() == str(w3.eth.accounts[0]).lower()

# check balance
assert w3.eth.getBalance(u_c_addr) == SEED_VALUE_BADPARITY

In [156]:
u_c_instance.getOwner()

'0xb1Cef50494b3C82Aa199DCedD212D333C9265697'

In [157]:
u_c_instance.getStudent()

'0x9DdBEe8f1Bb05273118d8e802797e5fB93Fc1BC0'

In [158]:
w3.parity.personal.listAccounts()

['0xb1Cef50494b3C82Aa199DCedD212D333C9265697',
 '0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0',
 '0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1']

In [159]:
# copy challenge metadata
challenge_path=CHALLENGES_PATH + "/badparity/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="badparity"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/166724aa9cc196ba721d86ef06fa61271301e011/badparity
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/badparity
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/badparity
/smartenv/setup/generated_config//attach/12c7b846d3b99747a82244cbac281d9bc914c9f3/badparity
/smartenv/setup/generated_config//attach/fcfb41d1d12f174606f35e1c9f63c38c7233b61c/badparity
/smartenv/setup/generated_config//attach/a48b5d4238c163368ccb060388638fb6d98a6ce9/badparity
/smartenv/setup/generated_config//attach/6b532e866ee6dbe941678081d08b2b874136c66a/badparity
/smartenv/setup/generated_config//attach/b4ebf9c576cac2b3c3caf17860bb70a6b02201f3/badparity
/smartenv/setup/generated_config//attach/fb327fb2321e3aa0e221d65758d0c790de01158c/badparity
/smartenv/setup/generated_config//attach/98970cac400e9b2194d4402064e77031c2ce11bd/badparity
/smartenv/setup/generated_config//attach/e9db33e3446894eace95a0fcf96b823be18e2e1

# Challenge: Daodown

In [160]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4'}

In [161]:
# !solc --overwrite -o "{CHALLENGES_PATH}/daodown/publish/" --abi --bin "{CHALLENGES_PATH}/daodown/publish/EDao.sol"

In [162]:
!solc_7.4 --overwrite -o "{CHALLENGES_PATH}/daodown/publish/" --abi --bin "{CHALLENGES_PATH}/daodown/publish/EDao.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges//daodown/publish/.


In [163]:
path=CHALLENGES_PATH + "/daodown/publish/EDao.sol"
challange_name="daodown"
seedvalue=SEED_VALUE_DAODOWN

forall_compile_and_deploy(users_list,
                          path,
                          challange_name,
                          seedvalue,
                          compiler="solc_7.4")
users_list[0]

Started for 19 users:
...................-------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [167]:
w3.eth.blockNumber

16004

In [168]:
w3.eth.getBlock(w3.eth.blockNumber)["gasUsed"]

0

In [166]:
w3.eth.getBlock(w3.eth.blockNumber-3)["gasUsed"]

2812700

In [169]:
path=CHALLENGES_PATH + "/daodown/publish/EDao.sol"
challange_name="daodown"
forall_check_contract(users_list,
                      path,
                      challange_name,
                      compiler="solc_7.4")
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 19 users:
...................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [170]:
# copy challenge metadata
challenge_path=CHALLENGES_PATH + "/daodown/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="daodown"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/166724aa9cc196ba721d86ef06fa61271301e011/daodown
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/daodown
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/daodown
/smartenv/setup/generated_config//attach/12c7b846d3b99747a82244cbac281d9bc914c9f3/daodown
/smartenv/setup/generated_config//attach/fcfb41d1d12f174606f35e1c9f63c38c7233b61c/daodown
/smartenv/setup/generated_config//attach/a48b5d4238c163368ccb060388638fb6d98a6ce9/daodown
/smartenv/setup/generated_config//attach/6b532e866ee6dbe941678081d08b2b874136c66a/daodown
/smartenv/setup/generated_config//attach/b4ebf9c576cac2b3c3caf17860bb70a6b02201f3/daodown
/smartenv/setup/generated_config//attach/fb327fb2321e3aa0e221d65758d0c790de01158c/daodown
/smartenv/setup/generated_config//attach/98970cac400e9b2194d4402064e77031c2ce11bd/daodown
/smartenv/setup/generated_config//attach/e9db33e3446894eace95a0fcf96b823be18e2e19/daodown
/smartenv/

## Challenge: Faildice

In [171]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [172]:
!ls -la /usr/local/bin/solc*

-rwxr-xr-x 1 root root 11833912 Jun 15 13:35 /usr/local/bin/solc
-rwxr-xr-x 1 root root  6284040 Feb 12  2019 /usr/local/bin/solc_5.4
-rwxr-xr-x 1 root root 11192752 Oct 19  2020 /usr/local/bin/solc_7.4
-rwxr-xr-x 1 root root 11833912 Apr 21  2021 /usr/local/bin/solc_8.4


In [173]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [174]:
!solc --overwrite -o "{CHALLENGES_PATH}/faildice/publish/" --abi --bin "{CHALLENGES_PATH}/faildice/publish/SatoshiFailDice.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges//faildice/publish/.


In [175]:
path= CHALLENGES_PATH + "/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
seedvalue=SEED_VALUE_FAILDICE

forall_compile_and_deploy(users_list,
                          path,
                          challange_name,
                          seedvalue,
                          argument2=True)
users_list[0]

Started for 19 users:
~~~~~~~~~~~~~~~~~~~-------------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [176]:
path= CHALLENGES_PATH + "/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 19 users:
...................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [177]:
# copy challenge metadata
challenge_path= CHALLENGES_PATH + "/faildice/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="faildice"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/166724aa9cc196ba721d86ef06fa61271301e011/faildice
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/faildice
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/faildice
/smartenv/setup/generated_config//attach/12c7b846d3b99747a82244cbac281d9bc914c9f3/faildice
/smartenv/setup/generated_config//attach/fcfb41d1d12f174606f35e1c9f63c38c7233b61c/faildice
/smartenv/setup/generated_config//attach/a48b5d4238c163368ccb060388638fb6d98a6ce9/faildice
/smartenv/setup/generated_config//attach/6b532e866ee6dbe941678081d08b2b874136c66a/faildice
/smartenv/setup/generated_config//attach/b4ebf9c576cac2b3c3caf17860bb70a6b02201f3/faildice
/smartenv/setup/generated_config//attach/fb327fb2321e3aa0e221d65758d0c790de01158c/faildice
/smartenv/setup/generated_config//attach/98970cac400e9b2194d4402064e77031c2ce11bd/faildice
/smartenv/setup/generated_config//attach/e9db33e3446894eace95a0fcf96b823be18e2e19/faildice

## Add optional/additional meta data

In [178]:
def add_login_token(path,users):
    login_tokens=list()
    with open(path,'r') as f_input:
        for line in f_input:
            #print(line.strip())
            login_tokens.append(line.strip())    
    
    for i,u in enumerate(users):
        u["login_token"] = login_tokens[i]
    
    return login_tokens.copy()

In [179]:
login_tokens = add_login_token(PARTICIPANTS_PATH + "login_tokens",users_list)

In [180]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [181]:
# login_tokens

## Write Students

In [182]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [183]:
import json

def persist_participants_json(path,users):
    with open(path,'w') as f_output:
        json.dump(users,f_output)

In [184]:
out_file_json = GEN_CONFIG_PATH + "/participants_meta.json"
persist_participants_json(out_file_json,users_list)
len(users_list)

19

In [185]:
import csv

def persist_participants_csv(path,
                   out_fieldnames=['firstname', 'lastname','ppid','email','cc','attach','contract','account'],
                   cc_mail="ajudmayer@sba-research.org"):
    with open(path,'w',encoding='utf-8') as f_output:
    #with open(path,'w',encoding='latin-1') as f_output:
        writer = csv.DictWriter(f_output, 
                                fieldnames=out_fieldnames, 
                                delimiter=',',
                                quotechar='\"',
                                extrasaction='ignore')
        writer.writeheader()
        for i,row in enumerate(users_list):
            row["cc"]=cc_mail
            #print(row)
            writer.writerow(row)
    return i+1

In [186]:
out_file = GEN_CONFIG_PATH + "/participants_meta.csv"
out_fieldnames = ['ppid','email','cc','attach','account']
#out_fieldnames = ['firstname', 'lastname','matr','email','cc','attach','contract','account']
#out_fieldnames = ['firstname', 
#                  'lastname',
#                  'matr',
#                  'email',
#                  'cc',
#                  'attach',
#                  'account',
#                  'notawallet_addr',
#                 ]
cc_mail = "ajudmayer@sba-research.org"
persist_participants_csv(out_file,out_fieldnames,cc_mail)

19

In [187]:
!head -n2 $out_file

In [188]:
!wc -l $out_file

20 /smartenv/setup/generated_config//participants_meta.csv


## Prepare mail

In [189]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x71334758a55585c7451f0df06c48cf18893f1f356f7e73e6c41050ff21067444',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x4c2549e53D28e76cC9FDe242115F0Cd247da786B',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xb163300ef679517e872f3d5a74ae60d924f06a365e7da9d75c2b244b091e1d30',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xCde8c8a5509557Fd9D3BA37a7064d81d023B4284',
 'badparity_lib': '0xc4b27c1b179Af861c1EF4F4A5743d666a18019B4',
 'daodown_tx': '0xc62cd2cf2d422cf096be4030f0bac3d8648b93487bf414aa20f476506a7a2114',
 'daodo

In [190]:
import os

def dump_addresses(users,attach_path=GEN_CONFIG_PATH + "/attach"):
    for u in users:
        arg_dest = attach_path + "/" + u["account"] + "/" + "addresses"
        ch_string = "notawallet_addr = " + u["notawallet_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "badparity_addr = " + u["badparity_addr"]
        !echo $ch_string >> $arg_dest    
        
        ch_string = "daodown_addr = " + u["daodown_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "faildice_addr = " + u["faildice_addr"]
        !echo $ch_string >> $arg_dest
        
        arg_dest = attach_path + "/" + u["account"] + "/" + "login_token"
        ch_string = "login_token = " + u["login_token"]
        !echo $ch_string >> $arg_dest
        
        print(".",end="")

In [191]:
# write all addresses to folders 
dump_addresses(users_list,attach_path=GEN_CONFIG_PATH + "/attach")

...................

In [192]:
# copy general information i.e., datadir to individual folders
source_path=GLOBAL_INFO_PATH
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name=""
copy_challenge_metadata(users_list,source_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/166724aa9cc196ba721d86ef06fa61271301e011/
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/
/smartenv/setup/generated_config//attach/12c7b846d3b99747a82244cbac281d9bc914c9f3/
/smartenv/setup/generated_config//attach/fcfb41d1d12f174606f35e1c9f63c38c7233b61c/
/smartenv/setup/generated_config//attach/a48b5d4238c163368ccb060388638fb6d98a6ce9/
/smartenv/setup/generated_config//attach/6b532e866ee6dbe941678081d08b2b874136c66a/
/smartenv/setup/generated_config//attach/b4ebf9c576cac2b3c3caf17860bb70a6b02201f3/
/smartenv/setup/generated_config//attach/fb327fb2321e3aa0e221d65758d0c790de01158c/
/smartenv/setup/generated_config//attach/98970cac400e9b2194d4402064e77031c2ce11bd/
/smartenv/setup/generated_config//attach/e9db33e3446894eace95a0fcf96b823be18e2e19/
/smartenv/setup/generated_config//attach/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479/
/sma

In [193]:
# create archives
import os

def create_user_archives(users,attach_path=GEN_CONFIG_PATH + "/attach",attach_tgz=GEN_CONFIG_PATH + "/attach_tgz"):
    !mkdir -p $attach_tgz
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_src_dir = attach_path + "/" + d 
            arg_dst_dir = attach_tgz + "/" + d + ".tar.gz"
            print(arg_dst_dir)
            #With verbose output:
            #!tar -cvzf $arg_dst_dir -C $arg_src_dir . 
            #Without verbose output
            !tar -czf $arg_dst_dir -C $arg_src_dir . 

In [194]:
attach_path=GEN_CONFIG_PATH + "/attach"
attach_tgz=GEN_CONFIG_PATH + "/attach_tgz"
create_user_archives(users_list,attach_path,attach_tgz)

/smartenv/setup/generated_config//attach_tgz/166724aa9cc196ba721d86ef06fa61271301e011.tar.gz
/smartenv/setup/generated_config//attach_tgz/ec783a2c453025391c4e70875c8cf4f34fa2ab1b.tar.gz
/smartenv/setup/generated_config//attach_tgz/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz
/smartenv/setup/generated_config//attach_tgz/12c7b846d3b99747a82244cbac281d9bc914c9f3.tar.gz
/smartenv/setup/generated_config//attach_tgz/fcfb41d1d12f174606f35e1c9f63c38c7233b61c.tar.gz
/smartenv/setup/generated_config//attach_tgz/a48b5d4238c163368ccb060388638fb6d98a6ce9.tar.gz
/smartenv/setup/generated_config//attach_tgz/6b532e866ee6dbe941678081d08b2b874136c66a.tar.gz
/smartenv/setup/generated_config//attach_tgz/b4ebf9c576cac2b3c3caf17860bb70a6b02201f3.tar.gz
/smartenv/setup/generated_config//attach_tgz/fb327fb2321e3aa0e221d65758d0c790de01158c.tar.gz
/smartenv/setup/generated_config//attach_tgz/98970cac400e9b2194d4402064e77031c2ce11bd.tar.gz
/smartenv/setup/generated_config//attach_tgz/e9db33e3446894eace95a0fcf

## Test loading a key

In [ ]:
#w3.eth.accounts

In [ ]:
#w3.geth.personal.unlockAccount(w3.eth.accounts[3],"INSERT",0)

In [ ]:
#w3.fromWei(w3.eth.getBalance(w3.eth.accounts[3]),'ether')

## Load Participants again and attach meta data

In [142]:
import json

def load_users(path):
    with open(path,'r') as f_input:
        return json.load(f_input)

In [143]:
users_loaded = load_users(GEN_CONFIG_PATH + "/participants_meta.json")
users_loaded[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo